In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from keras.layers import Embedding, Dense, LSTM, Activation, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
import pandas as pd
import numpy as np
import  seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [36]:
gmsc_train=pd.read_table('./Give Me Some Credit/cs-training.csv',sep=',')
gmsc_train=gmsc_train.drop(columns=['Unnamed: 0'])
gmsc_train.fillna(method='bfill',inplace=True)

In [37]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [38]:
gmsc_pre=gmsc_train.copy(deep=True)
gmsc_pre=gmsc_pre.drop(columns=['SeriousDlqin2yrs'])
pre_gmsc=StandardScaler().fit_transform(gmsc_pre)
pre_gmsc=pd.DataFrame(pre_gmsc,columns = gmsc_pre.select_dtypes(include='number').columns)
pre_gmsc=pd.concat([pre_gmsc, gmsc_train[['SeriousDlqin2yrs']]], axis=1)

In [39]:
from sklearn.ensemble import RandomForestClassifier
X, y = pre_gmsc.loc[:, np.delete(pre_gmsc.columns.values, 
                                        np.where(pre_gmsc.columns.values == ['SeriousDlqin2yrs']))], \
        pre_gmsc.loc[:, 'SeriousDlqin2yrs']

# train on randomForest to get important features
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
clf.fit(X, y) 

feature_importances = pd.DataFrame(sorted(zip(pre_gmsc.columns, clf.feature_importances_), key=lambda x: x[1] * -1),
                                    columns = ['feature','importance'])

In [40]:
top_features = feature_importances[feature_importances.importance>0]['feature'].values

X, y = pre_gmsc.loc[:,top_features], pre_gmsc.loc[:,'SeriousDlqin2yrs']

In [41]:
y=y*(-1)+1

In [42]:
from sklearn.model_selection import train_test_split, KFold,cross_val_score,StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE

In [43]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0,stratify = y)
sm = SMOTE(random_state=0,sampling_strategy=1)
x_train_b, y_train_b = sm.fit_resample(x_train, y_train.ravel())

In [11]:
from scipy.stats import ks_2samp

In [12]:
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic

In [13]:
def type2_calcu(y,yhat):
    confusion =confusion_matrix(y,yhat)
    #[row, column]
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    return FP / float(FP + TN)
def type1_calcu(y,yhat):
    confusion =confusion_matrix(y,yhat)
    #[row, column]
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    return FN / float(TP + FN)

In [14]:
def scores(model):
    train=[]
    test=[]
    y_train_c=model.predict(x_train)
    y_train_pre=model.predict_proba(x_train)
    y_pred_c=model.predict(x_test)
    y_pred = model.predict_proba(x_test)
    #roc
    train.append(roc_auc_score(y_train,y_train_pre))
    test.append(roc_auc_score(y_test, y_pred))
    #ks
    train.append(ks_stat(y_train,np.round(y_train_c[:,0],0)))
    test.append(ks_stat(y_test, np.round(y_pred_c[:,0],0)))
    #brier
    train.append(brier_score_loss(y_train,y_train_pre))
    test.append(brier_score_loss(y_test, y_pred))
    #acc
    train.append(accuracy_score(y_train,np.round(y_train_c,0)))
    test.append(accuracy_score(y_test, np.round(y_pred_c,0)))
    #t1
    train.append(type1_calcu(y_train,np.round(y_train_c,0)))
    test.append(type1_calcu(y_test, np.round(y_pred_c,0)))
    #t2
    train.append(type2_calcu(y_train,np.round(y_train_c,0)))
    test.append(type2_calcu(y_test, np.round(y_pred_c,0)))
    return train,test

In [15]:
print(x_train.shape)

(120000, 10)


In [44]:
x_train=StandardScaler().fit_transform(x_train)
x_test=StandardScaler().fit_transform(x_test)

x_train_b=StandardScaler().fit_transform(x_train_b)

In [45]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

x_train_b = x_train_b.reshape(x_train_b.shape[0], x_train_b.shape[1], 1)

In [18]:
epochs = 20
model = Sequential()
model.add(Conv1D(32, 2, activation='relu',input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64, 2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

Metal device set to: Apple M1 Pro


2022-07-27 21:08:08.374793: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-27 21:08:08.375081: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 9, 32)             96        
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 32)             128       
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 9, 32)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 8, 64)             4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 64)             256       
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 8, 64)             0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 512)               0

In [20]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [22]:
history = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
3750/3750 [==============================] - 33s 9ms/step - loss: 0.1975 - accuracy: 0.9345
Epoch 2/5
3750/3750 [==============================] - 33s 9ms/step - loss: 0.1952 - accuracy: 0.9351
Epoch 3/5
3750/3750 [==============================] - 32s 9ms/step - loss: 0.1946 - accuracy: 0.9354
Epoch 4/5
3750/3750 [==============================] - 32s 9ms/step - loss: 0.1931 - accuracy: 0.9355
Epoch 5/5
3750/3750 [==============================] - 32s 9ms/step - loss: 0.1924 - accuracy: 0.9353


In [23]:
from sklearn.metrics import roc_auc_score, recall_score, precision_score,make_scorer,confusion_matrix,brier_score_loss,accuracy_score

In [24]:
print(scores(model))

2022-07-27 21:13:20.674814: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
2022-07-27 21:13:31.314720: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


([0.8304612787952401, 0.16683116774797255, 0.056643497061568175, 0.9364416666666666, 0.0090820600291126, 0.8240867722229148], [0.8326065529157994, 0.1423733041035561, 0.056405843973303824, 0.9359666666666666, 0.007751384175745669, 0.8498753117206983])


In [25]:
model2 = Sequential()
model2.add(LSTM(32, input_shape=(1,10), activation='relu', return_sequences=True))
model2.add(Dropout(0.2))

model2.add(LSTM(64, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

In [26]:
x_train = x_train.reshape(x_train.shape[0], 1,x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1,x_test.shape[1])

In [27]:
x_train.shape

(120000, 1, 10)

In [28]:
model2.compile(optimizer=Adam(learning_rate=0.001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [29]:
history=model2.fit(x_train, y_train, epochs=5)

Epoch 1/5


2022-07-27 21:14:43.810946: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3750/3750 [==============================] - 61s 16ms/step - loss: 0.2058 - accuracy: 0.9339
Epoch 2/5
3750/3750 [==============================] - 60s 16ms/step - loss: 0.1911 - accuracy: 0.9346
Epoch 3/5
3750/3750 [==============================] - 60s 16ms/step - loss: 0.1905 - accuracy: 0.9354
Epoch 4/5
3750/3750 [==============================] - 59s 16ms/step - loss: 0.1898 - accuracy: 0.9356
Epoch 5/5
3750/3750 [==============================] - 63s 17ms/step - loss: 0.1893 - accuracy: 0.9362


In [30]:
print(scores(model2))

2022-07-27 21:19:46.045022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
2022-07-27 21:20:22.414216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


([0.8326426747697794, 0.2143209581951388, 0.05126713200652451, 0.9363666666666667, 0.012832763285973263, 0.7728462785188879], [0.8372003817924379, 0.19170612493591882, 0.05100260108651581, 0.9366333333333333, 0.010787640650116092, 0.7975062344139651])


In [31]:
epochs = 20
model3 = Sequential()
model3.add(Conv1D(32, 2, activation='relu',input_shape = x_train_b[0].shape))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

model3.add(Conv1D(64, 2, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))

model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.5))

model3.add(Dense(1, activation='sigmoid'))

In [46]:
x_train=x_train_b
y_train=y_train_b

In [47]:
model3.compile(optimizer=Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [48]:
history = model3.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
   1/6999 [..............................] - ETA: 1:20:44 - loss: 1.5463 - accuracy: 0.3438

2022-07-27 21:22:43.220142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


6999/6999 [==============================] - 131s 19ms/step - loss: 0.5852 - accuracy: 0.7036
Epoch 2/5
6999/6999 [==============================] - 131s 19ms/step - loss: 0.5251 - accuracy: 0.7418
Epoch 3/5
6999/6999 [==============================] - 130s 19ms/step - loss: 0.5136 - accuracy: 0.7506
Epoch 4/5
6999/6999 [==============================] - 130s 19ms/step - loss: 0.5081 - accuracy: 0.7544
Epoch 5/5
6999/6999 [==============================] - 130s 19ms/step - loss: 0.5036 - accuracy: 0.7581


In [49]:
print(scores(model3))

2022-07-27 21:33:34.816248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


([0.8431599485462711, 0.5202314719724234, 0.16392075293883182, 0.7601157359862117, 0.14684896275194456, 0.33291956527563205], [0.7643006878232175, 0.0014421884207145297, 0.5742499138278535, 0.0725, 0.9935702804072156, 0.004987531172069825])


In [65]:
model4 = Sequential()
model4.add(LSTM(32, input_shape=(1,10), activation='relu', return_sequences=True))
model4.add(Dropout(0.2))

model4.add(LSTM(64, activation='relu'))
model4.add(Dropout(0.5))

model4.add(Dense(64, activation='relu'))
model4.add(Dropout(0.5))

model4.add(Flatten())
model4.add(Dense(1, activation='sigmoid'))

In [51]:
#x_train = x_train.reshape(x_train.shape[0],1,x_train.shape[1])

In [69]:
x_test.shape

(30000, 10, 1)

In [70]:
#x_train=x_train_b
#y_train=y_train_b
x_test = x_test.reshape(x_test.shape[0], 1,x_test.shape[1])

In [66]:
model4.compile(optimizer=Adam(learning_rate=0.001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [67]:
history=model4.fit(x_train, y_train, epochs=5)

Epoch 1/5


2022-07-28 07:31:30.753968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


6999/6999 [==============================] - 142s 20ms/step - loss: 0.5246 - accuracy: 0.7422
Epoch 2/5
6999/6999 [==============================] - 140s 20ms/step - loss: 0.4985 - accuracy: 0.7633
Epoch 3/5
6999/6999 [==============================] - 140s 20ms/step - loss: 0.4918 - accuracy: 0.7677
Epoch 4/5
6999/6999 [==============================] - 141s 20ms/step - loss: 0.4846 - accuracy: 0.7746
Epoch 5/5
6999/6999 [==============================] - 140s 20ms/step - loss: 0.4765 - accuracy: 0.7792


In [71]:
print(scores(model4))

([0.8661577839397377, 0.5778404879486332, 0.15289471359273393, 0.7889202439743166, 0.2472517168397646, 0.17490779521160219], [0.8160745038635774, 0.0013957782806780505, 0.5795949228154921, 0.069, 0.997606715484908, 0.000997506234413965])
